In [1]:
from deep_orderbook.shaper import BookShaper, ArrayShaper
import asyncio
from pylab import rcParams
from tqdm.auto import tqdm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn_image as isns
from IPython.display import clear_output
# rcParams['figure.figsize'] = 20, 4

In [3]:
MARKETS = ["ETH-BTC", "BTC-USD", "ETH-USD"]

In [ ]:
from deep_orderbook.feeds.coinbase_feed import CoinbaseFeed
from deep_orderbook.replayer import ParquetReplayer

async def iter_sec(date_regexp='2024-08-06', max_samples=0):
    replayer = ParquetReplayer('data', date_regexp=date_regexp)
    async with CoinbaseFeed(
        markets=MARKETS,
        replayer=replayer,
    ) as feed:
        async for onesec in feed.one_second_iterator():
            yield onesec
            max_samples -= 1
            if max_samples == 0:
                break

async for onesec in iter_sec(max_samples=3):
    print(onesec)

In [ ]:
async def iter_shapes(date_regexp='2024-08-06', max_samples=0):
    symbol_shapers = {pair: ArrayShaper(zoom_frac=0.002) for pair in MARKETS}
    async for onesec in iter_sec(max_samples=max_samples):
        yield {
            market: await symbol_shapers[market].make_arr3d(
                onesec.symbols[market]
            )
            for market in MARKETS
        }

async for shaped in iter_shapes(max_samples=1):
    print(shaped)


In [ ]:
async def iter_gen_arrays(date_regexp='2024-08-06', max_samples=0):
    async for shapes in iter_shapes(max_samples=max_samples):
        yield shapes['ETH-USD'].transpose(1, 0, 2)
        # yield np.hstack([shapes[market] for market in MARKETS]).transpose(1, 0, 2)


async for arr in iter_gen_arrays(max_samples=100):
    im = arr.copy()
    im[:,:,0] *= -0.2
    im[:,:,1:3] *= 1e6
    isns.imshow(im, origin='lower', cmap='coolwarm', vmin=-1, vmax=1, gray=True)
    clear_output(wait=True)
    plt.show()


In [ ]:
async def iter_shapes_t2l(date_regexp='2024', max_samples=0):
    symbol_shaper = ArrayShaper(zoom_frac=0.002)
    async for onesec in iter_sec(max_samples=max_samples):
        yield [
            await symbol_shaper.make_arr3d(onesec.symbols['ETH-USD']),
            await symbol_shaper.build_time_level_trade(),
            symbol_shaper.prices_array
        ]

async for shapes in iter_shapes_t2l(max_samples=1):
    print(shapes[0].shape, shapes[1].shape, shapes[2].shape)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import asyncio
from IPython.display import display

# Define your asynchronous function to update the figure
async def update_figure():
    # Initialize the figure with three subplots and adjust row heights
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=("Books", "Level Proximity", "Bid and Ask Price Levels"),
        vertical_spacing=0.05,
        row_heights=[0.33, 0.33, 0.34]  # Adjust the relative heights of the rows
    )

    # Increase the overall figure height
    fig.update_layout(
        height=600,  # Adjusted for three subplots
        margin=dict(t=50, b=50, l=50, r=50),
        showlegend=True
    )

    # Create the figure widget
    fig_widget = go.FigureWidget(fig)
    display(fig_widget)

    # Initialize empty traces for heatmaps
    im_trace = go.Heatmap(
        z=np.zeros((10, 10)),
        colorscale='RdBu',
        zmin=-1, zmax=1,
        showscale=False
    )
    t2l_trace = go.Heatmap(
        z=np.zeros((10, 10)),
        colorscale='Turbo',
        zmin=0, zmax=1,
        showscale=False
    )

    # Initialize line traces for bid and ask price levels
    bid_trace = go.Scatter(
        x=[], y=[],
        mode='lines',
        line=dict(color='green'),
        showlegend=False
    )
    ask_trace = go.Scatter(
        x=[], y=[],
        mode='lines',
        line=dict(color='red'),
        showlegend=False
    )

    # Add traces to the figure widget
    fig_widget.add_trace(im_trace, row=1, col=1)
    fig_widget.add_trace(t2l_trace, row=2, col=1)
    fig_widget.add_trace(bid_trace, row=3, col=1)
    fig_widget.add_trace(ask_trace, row=3, col=1)

    # Asynchronous loop to update the figure
    async for shaped, t2l, pxar in iter_shapes_t2l(max_samples=1000):
        # Process your data for heatmaps
        im_data = shaped.copy().transpose(1, 0, 2)
        im_data[:, :, 0] *= -0.5
        im_data[:, :, 1:3] *= 1e6
        im_data = np.clip(im_data, -1, 1)
        t2l_data = np.clip(t2l[:, :, 0].T, -1, 1)

        # Extract bid and ask prices from pxar
        bid_prices = pxar[:, 0]
        ask_prices = pxar[:, 1]
        levels = np.arange(len(bid_prices))  # Assuming levels from 0 to 511

        # Update the figure widget traces
        with fig_widget.batch_update():
            # Update heatmaps
            fig_widget.data[0].z = im_data.mean(axis=2)
            fig_widget.data[1].z = t2l_data

            # Update bid and ask price traces
            fig_widget.data[2].x = levels
            fig_widget.data[2].y = bid_prices
            fig_widget.data[3].x = levels
            fig_widget.data[3].y = ask_prices

            # Update axes labels and ranges if necessary
            fig_widget.update_xaxes(title_text="Level", row=3, col=1)
            fig_widget.update_yaxes(title_text="Price", row=3, col=1)

        # Optional: add a small delay to control update rate
        await asyncio.sleep(0.05)

# Run the asynchronous function
await update_figure()

In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts
import panel as pn
from holoviews.streams import Pipe
from IPython.display import display
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Enable the Bokeh backend for HoloViews and initialize Panel
hv.extension('bokeh')
pn.extension()

# Initialize Streams with appropriate empty data
im_pipe = Pipe(data=np.zeros((10, 10)))        # For 'Books' heatmap
t2l_pipe = Pipe(data=np.zeros((10, 10)))       # For 'Level Proximity' heatmap
bidask_pipe = Pipe(data=np.empty((0, 2)))      # For 'Bid and Ask Price Levels'

# Define HoloViews DynamicMaps for heatmaps
im_dmap = hv.DynamicMap(lambda data: hv.Image(data).opts(
    cmap='RdBu', colorbar=True, width=600, height=200, title='Books',
    xaxis=None, yaxis=None
), streams=[im_pipe])

t2l_dmap = hv.DynamicMap(lambda data: hv.Image(data).opts(
    cmap='Turbo', colorbar=True, width=600, height=200, title='Level Proximity',
    xaxis=None, yaxis=None
), streams=[t2l_pipe])

# Define HoloViews DynamicMap for Bid and Ask Prices
def bidask_plot(data):
    bid_curve = hv.Curve((data[:, 0]), 'bid').opts(color='green', line_width=2, show_legend=True)#, framewise=True)
    ask_curve = hv.Curve((data[:, 1]), 'ask').opts(color='red', line_width=2, show_legend=True)#, framewise=True)
    return (bid_curve * ask_curve).opts(width=600, height=200, title='Bid and Ask', 
                                        yaxis=None, xaxis='bottom')

bidask_dmap = hv.DynamicMap(bidask_plot, streams=[bidask_pipe])

# Layout the plots vertically using Panel
dashboard = pn.Column(
    pn.panel(im_dmap),
    pn.panel(t2l_dmap),
    pn.panel(bidask_dmap)
)

# Display the dashboard
display(dashboard)

# Define the async update function
async def run_updates():
    async for shaped, t2l, pxar in iter_shapes_t2l(max_samples=100):
        if np.isnan(pxar).any().any():
            pxar[np.isnan(pxar)] = np.nanmean(pxar)
        # Process the 'Books' heatmap data
        im_data = shaped.copy().transpose(1, 0, 2)
        im_data[:, :, 0] *= -0.5
        im_data[:, :, 1:3] *= 1e6
        im_data = np.clip(im_data, -1, 1)
        im_mean = im_data.mean(axis=2)

        # Process the 'Level Proximity' heatmap data
        t2l_data = np.clip(t2l[:, :, 0].T, -1, 1)

        # Send data through the streams
        im_pipe.send(im_mean)
        t2l_pipe.send(t2l_data)
        bidask_pipe.send(pxar)  # Pass pxar directly

# Start the async update in the background
asyncio.create_task(run_updates())
1


In [ ]:
import altair as alt
import pandas as pd
import plotly.express as px

source = pd.DataFrame({"t": np.arange(pxar.shape[0]), "p": pxar[:,0]})
chart = alt.Chart(source).encode()
line = chart.mark_line().encode(x="t:Q", y="p:Q")
line

px.imshow(t2l, origin="lower", zmin=0, zmax=150)

In [ ]:
import plotly.graph_objects as go
import numpy as np
import ipywidgets as iw
fw = go.FigureWidget()

fw.add_trace(go.Bar(x=[1, 2, 3], y=[1, 3, 2]))

N=10
test_data = np.random.randint(2, 8, (3, N))
slider = iw.IntSlider(value=1, min=1, max=N, step=1, description='Step')
slider.layout = dict(margin='12px 80px 40px 5px', width='700px')
def step_changed(change):
    fw.data[0].y = test_data[:, slider.value-1]
    
slider.observe(step_changed, 'value')  
play_button = iw.Play(value=1, min=1, max=N,  interval=225)
play_button.layout = dict(margin='12px 10px 50px 100px')
iw.link((play_button, 'value'), (slider, 'value'))
iw.VBox([fw, iw.HBox([play_button, slider])])#from slider to plot

In [ ]:
import plotly.express as px
from skimage import io
img = io.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/0/00/Crab_Nebula.jpg/240px-Crab_Nebula.jpg')
fig = px.imshow(img)
fig.show()

In [ ]:
# library imports
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh', logo=False)
import panel as pn
import time

# create sample data
data = np.random.normal(size=[50, 2])
df = pd.DataFrame(data, columns=['col1', 'col2'])

# create holoviews graph
hv_plot = hv.Image(im[:,:,0], bounds=(0, 0, 1, 1))
# hv_plot.data = im[:,:,1]

# display graph in browser
# a bokeh server is automatically started
bokeh_server = pn.Row(hv_plot).show(port=12345)

await asyncio.sleep(1)


In [ ]:

# stop the bokeh server (when needed)
bokeh_server.stop()

In [ ]:
import holoviews as hv

hv.extension("bokeh", logo=False)

import numpy as np
import asyncio
import panel as pn

pn.extension()
import nest_asyncio

nest_asyncio.apply()


# Create a reactive HoloViews stream
stream = hv.streams.Stream.define('ImageUpdate', data=None)()


# Function to update the image
async def update_image():
    async for arr in iter_gen_arrays(max_samples=-1):
        im = arr.copy()
        im[:, :, 0] *= -0.5
        im[:, :, 1:3] *= 1e6
        im_display = im.mean(axis=2)

        # Update the stream with the processed image
        stream.event(data=im_display)
        # await asyncio.sleep(0.1)


# Start the asynchronous update
asyncio.create_task(update_image())


# Function to create the hv.Image with correct bounds
def create_image(data):
    if data is not None:
        height, width = data.shape
        return hv.Image(data, bounds=(0, 0, width, height))
    else:
        # Placeholder image if data is None
        return hv.Image(np.zeros((128, 256)), bounds=(0, 0, 256, 128))


# DynamicMap that updates with the stream
dmap = hv.DynamicMap(create_image, streams=[stream])

# Set display options
dmap.opts(
    cmap='RdBu',
    frame_width=800,
    aspect=256 / 128,
    invert_yaxis=True,
    xlabel='second',
    ylabel='levels',
    colorbar=True,
    clim=(-1, 1),
)

# Display with Panel
pn.panel(dmap).show()